In [ ]:
import cv2
import numpy as np

def nothing(x):
    pass

# Step 1: Load image
img = cv2.imread(r"C:\Users\gajal\OneDrive\Pictures\download.jpeg")
clone = img.copy()

# Step 2: Select face ROI
r = cv2.selectROI("Select Face ROI", img)
x, y, w, h = map(int, r)
roi = clone[y:y+h, x:x+w]

# Step 3: Create window + trackbars
cv2.namedWindow("ROI Window",cv2.WINDOW_FREERATIO)
cv2.createTrackbar("H", "ROI Window", 170, 179, nothing)
cv2.createTrackbar("S", "ROI Window", 150, 255, nothing)
cv2.createTrackbar("V", "ROI Window", 150, 255, nothing)
cv2.createTrackbar("Blur", "ROI Window", 5, 50, nothing)

# Step 4: Draw lips polygon
points = []
temp = roi.copy()

print("Left-click: add points around lips | Right-click: undo | Press ENTER when done")

def draw_points(event, x1, y1, flags, param):
    global points, temp
    if event == cv2.EVENT_LBUTTONDOWN:
        points.append((x1, y1))
    elif event == cv2.EVENT_RBUTTONDOWN and points:
        points.pop()

cv2.setMouseCallback("ROI Window", draw_points)

while True:
    temp = roi.copy()
    if len(points) > 1:
        cv2.polylines(temp, [np.array(points)], isClosed=True, color=(0,255,255), thickness=1)
    for p in points:
        cv2.circle(temp, p, 2, (0,255,255), -1)
    cv2.imshow("ROI Window", temp)

    key = cv2.waitKey(1) & 0xFF
    if key == 13:  # Enter key
        break

# Create lip mask
mask = np.zeros(roi.shape[:2], np.uint8)
cv2.fillPoly(mask, [np.array(points)], 255)

# Step 5: Apply tint using trackbars
while True:
    h_val = cv2.getTrackbarPos("H", "ROI Window")
    s_val = cv2.getTrackbarPos("S", "ROI Window")
    v_val = cv2.getTrackbarPos("V", "ROI Window")
    blur_val = cv2.getTrackbarPos("Blur", "ROI Window")

    if blur_val < 1: blur_val = 1
    if blur_val % 2 == 0: blur_val += 1

    roi_copy = roi.copy()
    roi_copy = cv2.GaussianBlur(roi_copy, (blur_val, blur_val), 0)

    # Convert HSV to BGR for tint color
    hsv_color = np.uint8([[[h_val, s_val, v_val]]])
    bgr_color = cv2.cvtColor(hsv_color, cv2.COLOR_HSV2BGR)[0][0]

    # Create tint and apply only inside mask
    tint = np.full_like(roi_copy, bgr_color)
    tinted = cv2.addWeighted(roi_copy, 0.6, tint, 0.4, 0)
    final_roi = np.where(mask[:,:,None]==255, tinted, roi_copy)

    # Replace in full image
    output = clone.copy()
    output[y:y+h, x:x+w] = final_roi

    combined = np.hstack((clone, output))
    cv2.imshow("ROI Window", final_roi)
    cv2.imshow("Before | After", combined)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.imwrite("virtual_makeup_polygon.jpg", output)
cv2.destroyAllWindows()